In [1]:
import sys
sys.path.append('../scratch_2')
from common.np import *
from common.layers import Embedding, SigmoidWithLoss
import collections 

import numpy as np

### Embedding 계층의 구현

입력층과 은닉층 사이의 가중치과 입력값의 Matmul의 결과는 가중치 행렬의 row vector를 끄집어 내는것과 같은 일을 하므로 행렬곱으로 표현하지 않고, 행렬의 일부를 가지고 오는 slicing, indexing을 이용할 수 있다.

In [2]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]            # MatMul 연산에서 바뀜
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0

        # dW[self.idx] = dout
        # 이렇게 하면 배열의 원소중 값이 값은 원소가 있다면 할당시 문제가 생김.
        # 따라서 할당이 아닌 더하기를 해준다.
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        
        return None

Embedding 계층을 이용하면 입력층 계산에서의 낭비를 줄일 수 있다. 남은 문제는 은닉층 이후의 처리이다. 은닉층 이후에서 계산이 오래걸리는 곳은 다음의 두 부분 이다.
* 은닉층의 뉴런과 가중치 행렬(Wout)의 곱
* Softmax계층의 계산
앞서 언급한 두가지는 모두 어휘가 많아지면 계산량이 매우 증가한다.

따라서 Softmax를 대신할 가벼운 계산이 필요한데 이것이 *네거티브 샘플링*이다.

### 네거티브 샘플링 기법

다중분류(multi - classification)을 이진 분류(binary - classification)로 근사하는것이 네거티브 샘플링이다.
* 다중분류는 지금까지 한것과 같이 여러개의 단어 중에서 옳은 단어 하나를 선택하는 것을 말한다.
* 이진 분류는 Yes/No로 대답하는 것과 같은데, 타깃 단어는 say입니까? 와 같은 질문에 답하는 형식이다.

이렇게 하면 충력층에는 뉴런을 하나만 준비하면 된다. 이 뉴런이 점수(얼마나 yes인지)를 출력하는 것이다. 여기서 시그모이드 함수를 통해 확률로 변환한다.(시그모이드 함수는 0부터 1까지의 출력을 갖는데 이를 확률로 해석한다.)

cross entropy error를 loss함수로 사용하는데, cross entropy error와 sigmoid함수의 역전파를 계산하면 출력이 y-t즉 오차가 된다. 이 오차는 앞의 계층으로 흘러가게 된다. 즉, 오차가 크면 크게 학습하고 오차가 작으면 작게 학습한다는 의미이다.

In [8]:
class EmbeddingDot:
    def __init__(self, W): # 가중치 행렬
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx): # 은닉층, 단어 번호
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis = 1)
       
        self.cache = (h, target_W)
        return out
    
    def backward(self,dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout*target_W
        return dh

우리가 EmbeddingDot 계층과 sigmoid계층을 을 통해 원하는 것은 정답일경우 sigmoid의 출력을 1에 가깝게 하고, 오답일 경우 sigmoid의 출력을 0에 가깝게 하는 것이다. 이를 위해서는 정답일경우 순전파의 출력이 크도록, 오답일 경우 순전파의 출력이 작도록 만들어야 한다.

그럼 오답에 대해서도 학습을 진행해야 하는데 이렇게 되면 어휘수가 많아질수록 계산량이 매우 커진다는 같은 문제점을 접하게 된다. 때문에 오답의 일부만 샘플링 하여 학습에 사용할것이다. 이를 negative sampling 이라고 한다.

정리하자면, 정답을 타깃으로 한경우 손실함수를 계산하고, 오답을 타깃으로 한경우 손실함수를 계산하여 이들을 더한것을 최종 손실로 생각한다.

그럼 샘플링 하는 방식이 중요해진다.가장 간단한 방식은 당연히 무작위로 샘플링 하는 것이나, 더 좋은 방법은 말뭉치의 통계 데이터를 활용하는 방법이다. 즉, 말뭉치에 많이 등장하는 단어는 자주 샘플링하고, 적게 등장하는 단어는 적게 샘플링하는 것이다. 이를 위해서는 각 단어의 확률분포를 알아야 한다.

np.random.choice()를 무작위 샘플링 용도로 사용할 수 있다. p에 확률분포를 담은 리스트를 지정하고, replace=False로 설정하면 네거티브 샘플링 용도로 활용 할수 있게 된다. 

하지만 네거티브 샘플링에서는 기본 확률분포에 0.75를 곱하는데 이는 확률분포가 낮은 단어를 버리지 않기 위해서 이다. 0.75제곱을 하면 확률분포가 낮은 단어의 확률을 살짝 높일 수 있다.

In [10]:
class UnigramSampler:
    def __init__(self, corpus, power, sample_size): # 말뭉치, 확률분포 변경을 위한 지수, 네거티브 샘플링 시행 횟수
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None
        
        # 말뭉치에 포함된 각 단어의 개수 세기
        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1
        
        # 말뭉치속 단어의 종류 
        vocab_size = len(counts)
        self.vocab_size = vocab_size
        
        # 확률분포를 표현할 리스트 0으로 초기화
        self.word_p = np.zeros(vocab_size)
        
        # 각 단어의 확률 분포 계산, 낮은 확률의 가지는 단어를 버리지 않기 위해 power 제곱
        for i in range(vocab_size):
            self.word_p[i] = counts[i]
            
        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
        
    def get_negative_sample(self, target): # target 단어를 긍정적 예로 해석하고 그외의 단어 ID를 샘플링 -> 부적적 샘플링 시행
        batch_size = target.shape[0]
        
        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype = np.int32)
            
            for i in range(batch_size):
                p = self.word_p.copy() # 원본 리스트를 건드리지 않음. 똑같이 생긴 다른 리스트를 리턴함
                target_idx = target[i] # 미니배치중 i번째로 받은 단어를 target으로 설정
                p[target_idx] = 0      # target은 뽑히면 안됨.
                p /= p.sum()
                
                negative_sample[i,:] = np.random.choice(self.vocab_size, size = self.sample_size, replace=False, p=p)
        
        else :
            print("GPU - may contain target word")
            # GPU로 계산할때는 속도를 우선하기 위해 target을 포함할수도 있게됨
            
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size), replace = True, p = self.word_p)
        
        return negative_sample

In [6]:
corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size=2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

not GPU
[[4 3]
 [0 2]
 [3 1]]


In [5]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power = 0.75, sample_size = 5):
        self.sample_size = sample_size
        self.sampler  = UnigramSampler(corpus, power, sample_size)
        
        # 부적정예를 다루는 계층 sample_size개  + 긍정적 예를 다루를 계층 1개
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1 )] 
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self.grads = [],[]
        
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32) # 정답 레이블
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 부정적예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32) # 정답 레이블
        for i in range(self.sample_size):
            negative_target = negative_sample[:,i]                           # i번째 coluumn
            score = self.embed_dot_layers[1 + i].forward(h, negative_target) # 각 계츠의 순전파 호출
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
        
    
    def backward(self, dout = 1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh  += l1.backward(dscore)
            
        return dh
        
        
        
        

### CBOW 모델

In [6]:
import sys
sys.path.append('../from_scratch2')
import numpy as np

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # 계층 생성
        self.in_layers = []
        for i in range(2*window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        # 모든 가중치과 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 인스턴스 변수에 단어의 분산표현을 저장한다.
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:,i])
        h *= 1/len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout = 1):
        dout = self.ns_loss.backward(dout)
        dout *= 1/len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

CBOW모델의 학습용 코드

In [ ]:
from common import config
# config.GPU = True
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
# from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)
    
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)

params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 3[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 4[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 5[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 6[s] | 손실 3.94
| 에폭 1 |  반복 121 / 9295 | 시간 8[s] | 손실 3.79
| 에폭 1 |  반복 141 / 9295 | 시간 9[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 10[s] | 손실 3.50
| 에폭 1 |  반복 181 / 9295 | 시간 12[s] | 손실 3.36
| 에폭 1 |  반복 201 / 9295 | 시간 13[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 14[s] | 손실 3.17
| 에폭 1 |  반복 241 / 9295 | 시간 16[s] | 손실 3.08
| 에폭 1 |  반복 261 / 9295 | 시간 17[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 18[s] | 손실 2.95
| 에폭 1 |  반복 301 / 9295 | 시간 20[s] | 손실 2.91
| 에폭 1 |  반복 321 / 9295 | 시간 21[s] | 손실 2.89
| 에폭 1 |  반복 341 / 9295 | 시간 22[s] | 손실 2.82
| 에폭 1 |  반복 361 / 9295 | 시간 23[s] | 손실 2.83
| 에폭 1 |  반복 381 / 9295 | 시간 25[s] | 손실 2.79
| 에폭 1 |  반복 401 / 9295 | 시간 26[s] | 손실 2.75
| 에폭 1 |  반복 421 / 9295 | 시간 27[s] | 손실 2.75
| 에폭 1 |  반복 441 / 9295 

| 에폭 1 |  반복 3561 / 9295 | 시간 245[s] | 손실 2.42
| 에폭 1 |  반복 3581 / 9295 | 시간 246[s] | 손실 2.42
| 에폭 1 |  반복 3601 / 9295 | 시간 247[s] | 손실 2.39
| 에폭 1 |  반복 3621 / 9295 | 시간 249[s] | 손실 2.45
| 에폭 1 |  반복 3641 / 9295 | 시간 250[s] | 손실 2.40
| 에폭 1 |  반복 3661 / 9295 | 시간 252[s] | 손실 2.40
| 에폭 1 |  반복 3681 / 9295 | 시간 253[s] | 손실 2.41
| 에폭 1 |  반복 3701 / 9295 | 시간 254[s] | 손실 2.40
| 에폭 1 |  반복 3721 / 9295 | 시간 256[s] | 손실 2.41
| 에폭 1 |  반복 3741 / 9295 | 시간 257[s] | 손실 2.42
| 에폭 1 |  반복 3761 / 9295 | 시간 259[s] | 손실 2.42
| 에폭 1 |  반복 3781 / 9295 | 시간 260[s] | 손실 2.42
| 에폭 1 |  반복 3801 / 9295 | 시간 261[s] | 손실 2.44
| 에폭 1 |  반복 3821 / 9295 | 시간 263[s] | 손실 2.39
| 에폭 1 |  반복 3841 / 9295 | 시간 264[s] | 손실 2.44
| 에폭 1 |  반복 3861 / 9295 | 시간 266[s] | 손실 2.39
| 에폭 1 |  반복 3881 / 9295 | 시간 267[s] | 손실 2.41
| 에폭 1 |  반복 3901 / 9295 | 시간 268[s] | 손실 2.40
| 에폭 1 |  반복 3921 / 9295 | 시간 270[s] | 손실 2.38
| 에폭 1 |  반복 3941 / 9295 | 시간 271[s] | 손실 2.39
| 에폭 1 |  반복 3961 / 9295 | 시간 272[s] | 손실 2.38
| 에폭 1 |  반복 

| 에폭 1 |  반복 7061 / 9295 | 시간 494[s] | 손실 2.28
| 에폭 1 |  반복 7081 / 9295 | 시간 495[s] | 손실 2.28
| 에폭 1 |  반복 7101 / 9295 | 시간 496[s] | 손실 2.26
| 에폭 1 |  반복 7121 / 9295 | 시간 498[s] | 손실 2.27
| 에폭 1 |  반복 7141 / 9295 | 시간 499[s] | 손실 2.33
| 에폭 1 |  반복 7161 / 9295 | 시간 501[s] | 손실 2.29
| 에폭 1 |  반복 7181 / 9295 | 시간 502[s] | 손실 2.26
| 에폭 1 |  반복 7201 / 9295 | 시간 503[s] | 손실 2.26
| 에폭 1 |  반복 7221 / 9295 | 시간 505[s] | 손실 2.24
| 에폭 1 |  반복 7241 / 9295 | 시간 506[s] | 손실 2.27
| 에폭 1 |  반복 7261 / 9295 | 시간 508[s] | 손실 2.28
| 에폭 1 |  반복 7281 / 9295 | 시간 509[s] | 손실 2.28
| 에폭 1 |  반복 7301 / 9295 | 시간 511[s] | 손실 2.28
| 에폭 1 |  반복 7321 / 9295 | 시간 512[s] | 손실 2.33
| 에폭 1 |  반복 7341 / 9295 | 시간 513[s] | 손실 2.27
| 에폭 1 |  반복 7361 / 9295 | 시간 515[s] | 손실 2.30
| 에폭 1 |  반복 7381 / 9295 | 시간 516[s] | 손실 2.24
| 에폭 1 |  반복 7401 / 9295 | 시간 518[s] | 손실 2.25
| 에폭 1 |  반복 7421 / 9295 | 시간 519[s] | 손실 2.29
| 에폭 1 |  반복 7441 / 9295 | 시간 520[s] | 손실 2.26
| 에폭 1 |  반복 7461 / 9295 | 시간 522[s] | 손실 2.27
| 에폭 1 |  반복 

| 에폭 2 |  반복 1281 / 9295 | 시간 737[s] | 손실 2.08
| 에폭 2 |  반복 1301 / 9295 | 시간 738[s] | 손실 2.13
| 에폭 2 |  반복 1321 / 9295 | 시간 740[s] | 손실 2.14
| 에폭 2 |  반복 1341 / 9295 | 시간 741[s] | 손실 2.15
| 에폭 2 |  반복 1361 / 9295 | 시간 742[s] | 손실 2.15
| 에폭 2 |  반복 1381 / 9295 | 시간 744[s] | 손실 2.10
| 에폭 2 |  반복 1401 / 9295 | 시간 745[s] | 손실 2.14
| 에폭 2 |  반복 1421 / 9295 | 시간 746[s] | 손실 2.14
| 에폭 2 |  반복 1441 / 9295 | 시간 748[s] | 손실 2.10
| 에폭 2 |  반복 1461 / 9295 | 시간 749[s] | 손실 2.14
| 에폭 2 |  반복 1481 / 9295 | 시간 750[s] | 손실 2.15
| 에폭 2 |  반복 1501 / 9295 | 시간 751[s] | 손실 2.13
| 에폭 2 |  반복 1521 / 9295 | 시간 753[s] | 손실 2.15
| 에폭 2 |  반복 1541 / 9295 | 시간 754[s] | 손실 2.14
| 에폭 2 |  반복 1561 / 9295 | 시간 755[s] | 손실 2.16
| 에폭 2 |  반복 1581 / 9295 | 시간 757[s] | 손실 2.15
| 에폭 2 |  반복 1601 / 9295 | 시간 758[s] | 손실 2.17
| 에폭 2 |  반복 1621 / 9295 | 시간 759[s] | 손실 2.13
| 에폭 2 |  반복 1641 / 9295 | 시간 761[s] | 손실 2.16
| 에폭 2 |  반복 1661 / 9295 | 시간 762[s] | 손실 2.18
| 에폭 2 |  반복 1681 / 9295 | 시간 763[s] | 손실 2.14
| 에폭 2 |  반복 

| 에폭 2 |  반복 4781 / 9295 | 시간 988[s] | 손실 2.10
| 에폭 2 |  반복 4801 / 9295 | 시간 989[s] | 손실 2.08
| 에폭 2 |  반복 4821 / 9295 | 시간 991[s] | 손실 2.07
| 에폭 2 |  반복 4841 / 9295 | 시간 992[s] | 손실 2.10
| 에폭 2 |  반복 4861 / 9295 | 시간 994[s] | 손실 2.09
| 에폭 2 |  반복 4881 / 9295 | 시간 996[s] | 손실 2.12
| 에폭 2 |  반복 4901 / 9295 | 시간 997[s] | 손실 2.05
| 에폭 2 |  반복 4921 / 9295 | 시간 999[s] | 손실 2.09
| 에폭 2 |  반복 4941 / 9295 | 시간 1000[s] | 손실 2.09
| 에폭 2 |  반복 4961 / 9295 | 시간 1002[s] | 손실 2.11
| 에폭 2 |  반복 4981 / 9295 | 시간 1003[s] | 손실 2.14
